In [12]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import json
from PIL import Image

# Define paths
images_folder = 'C:/Users/cim-security/Desktop/Obj/Low_lit/images'
labels_folder = 'C:/Users/cim-security/Desktop/Obj/Low_lit/Label'
output_file = 'C:/Users/cim-security/Desktop/Obj/Low_lit/annotate/annotations.json'

# Initialize COCO format data
coco_format = {
    'images': [],
    'annotations': [],
    'categories': [{'id': i + 1, 'name': name} for i, name in enumerate(class_names)]
}

annotation_id = 1

for image_file in os.listdir(images_folder):
    if image_file.endswith(('jpg', 'jpeg', 'png')):
        image_path = os.path.join(images_folder, image_file)
        label_path = os.path.join(labels_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Load image
        with Image.open(image_path) as img:
            width, height = img.size

        # Add image entry
        coco_format['images'].append({
            'id': len(coco_format['images']) + 1,
            'file_name': image_file,
            'width': width,
            'height': height
        })

        # Read and parse label file
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_name = parts[0]
                    if class_name in class_id_map:
                        # Skip 'Misc' and 'DontCare' if they should not be included
                        if class_name in ['Misc', 'DontCare']:
                            continue

                        class_id = class_id_map[class_name]
                        # Parse bounding box coordinates (example format: x_center, y_center, width, height)
                        x_center, y_center, w, h = map(float, parts[1:])
                        # Convert to [x, y, width, height] format
                        x = x_center - (w / 2)
                        y = y_center - (h / 2)

                        coco_format['annotations'].append({
                            'id': annotation_id,
                            'image_id': len(coco_format['images']),
                            'category_id': class_id,
                            'bbox': [x, y, w, h],
                            'area': w * h,
                            'iscrowd': 0
                        })

                        annotation_id += 1
                    else:
                        print(f"Warning: Class name '{class_name}' not found in class_id_map")

# Save to JSON file
with open(output_file, 'w') as f:
    json.dump(coco_format, f, indent=4)

print(f'Annotations saved to {output_file}')

Annotations saved to C:/Users/cim-security/Desktop/Obj/Low_lit/annotate/annotations.json
